In [1]:
!pip install Faker

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 32.2 MB/s eta 0:00:00a 0:00:01


In [2]:
!pip list

Package                            Version              Editable project location
---------------------------------- -------------------- -------------------------
absl-py                            1.4.0
accelerate                         1.2.1
aiofiles                           22.1.0
aiohappyeyeballs                   2.4.6
aiohttp                            3.11.12
aiosignal                          1.3.2
aiosqlite                          0.21.0
alabaster                          1.0.0
albucore                           0.0.19
albumentations                     1.4.20
alembic                            1.14.1
altair                             5.5.0
annotated-types                    0.7.0
annoy                              1.17.3
ansicolors                         1.1.8
antlr4-python3-runtime             4.9.3
anyio                              3.7.1
argon2-cffi                        23.1.0
argon2-cffi-bindings               21.2.0
args                               0.1.0
array_

In [3]:
!pip install accelerate

<h2>Data Analysis</h2>

Generate a synthetic dataset where each entry represents a book read by a user

In [10]:
import random
import pandas as pd
from faker import Faker

fake = Faker()

# Parameters
num_users = 10
min_books_per_user = 3
max_books_per_user = 7
genres = ["Fantasy", "Romance", "Sci-Fi", "Mystery", "Thriller", "Adventure"]
days_in_month = 30

# Generate users
users = []
for i in range(num_users):
    user_id = f"user_{i+1}"
    given_name = fake.first_name()
    users.append((user_id, given_name))

# Generate dataset
data = []
book_id_counter = 1

for user_id, given_name in users:
    num_books = random.randint(min_books_per_user, max_books_per_user)
    for _ in range(num_books):
        book_title = fake.sentence(nb_words=3).rstrip('.')
        book_id = f"book_{book_id_counter}"
        book_id_counter += 1
        genre = random.choice(genres)
        # Simulate days user accessed this book
        access_days = random.sample(range(1, days_in_month + 1), random.randint(3, 15))
        for day in access_days:
            data.append({
                "user_id": user_id,
                "given_name": given_name,
                "book_title": book_title,
                "book_id": book_id,
                "genre": genre,
                "day": day
            })

# Create DataFrame and sort by user and day
df = pd.DataFrame(data)
df = df.sort_values(by=["user_id", "day"]).reset_index(drop=True)

df.head()


,user_id,given_name,book_title,book_id,genre,day
0,user_1,Brianna,Entire throw reveal,book_6,Adventure,1
1,user_1,Brianna,Entire throw reveal,book_6,Adventure,3
2,user_1,Brianna,Add,book_1,Adventure,4
3,user_1,Brianna,Past apply,book_3,Sci-Fi,4
4,user_1,Brianna,Throughout often,book_5,Fantasy,4


In [11]:
df.groupby("book_id").head()

,user_id,given_name,book_title,book_id,genre,day
0,user_1,Brianna,Entire throw reveal,book_6,Adventure,1
1,user_1,Brianna,Entire throw reveal,book_6,Adventure,3
2,user_1,Brianna,Add,book_1,Adventure,4
3,user_1,Brianna,Past apply,book_3,Sci-Fi,4
4,user_1,Brianna,Throughout often,book_5,Fantasy,4
...,...,...,...,...,...,...
462,user_9,Barbara,Kitchen fact hear,book_40,Sci-Fi,15
467,user_9,Barbara,North church,book_39,Thriller,18
468,user_9,Barbara,Kitchen fact hear,book_40,Sci-Fi,19
475,user_9,Barbara,North church,book_39,Thriller,25


Find the top genre read by each user

In [12]:
genre_counts = df.groupby(['user_id', 'genre', 'given_name']).size().reset_index(name='count')
top_genres = genre_counts.sort_values('count', ascending=False).drop_duplicates('user_id')
top_genres

,user_id,genre,given_name,count
5,user_10,Mystery,Thomas,33
17,user_5,Adventure,Holly,27
15,user_4,Adventure,Jared,24
0,user_1,Adventure,Brianna,23
22,user_6,Romance,Michael,21
14,user_3,Romance,Marc,21
11,user_2,Romance,Alexandra,21
27,user_7,Thriller,Mary,18
31,user_9,Fantasy,Barbara,15
29,user_8,Fantasy,Daniel,13


Check which users have a 30 day streak

In [13]:
# Check if each user has all 30 days of activity
complete_streak_users = df.groupby(['user_id', 'given_name'])['day'].apply(lambda x: set(range(1, 31)).issubset(set(x)))

complete_streak_users = complete_streak_users.reset_index(name='streak')

print(complete_streak_users)


   user_id given_name  streak
0   user_1    Brianna   False
1  user_10     Thomas   False
2   user_2  Alexandra   False
3   user_3       Marc   False
4   user_4      Jared   False
5   user_5      Holly   False
6   user_6    Michael    True
7   user_7       Mary   False
8   user_8     Daniel   False
9   user_9    Barbara   False


Join both dataframes on user_id and name

In [14]:
# join top genres and complete_streak_users
trend = top_genres.merge(
    complete_streak_users,
    on=['user_id', 'given_name']
)
trend.head(10)

,user_id,genre,given_name,count,streak
0,user_10,Mystery,Thomas,33,False
1,user_5,Adventure,Holly,27,False
2,user_4,Adventure,Jared,24,False
3,user_1,Adventure,Brianna,23,False
4,user_6,Romance,Michael,21,True
5,user_3,Romance,Marc,21,False
6,user_2,Romance,Alexandra,21,False
7,user_7,Thriller,Mary,18,False
8,user_9,Fantasy,Barbara,15,False
9,user_8,Fantasy,Daniel,13,False


In [17]:
selected_users = trend[trend["streak"]==True]
selected_users.iloc[0]["genre"]

'Romance'

<h2>Story generation section<h2>

Initialize LLM

In [23]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
model_name = "Qwen/Qwen2.5-7B-Instruct-1M"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

config.json:   0%|          | 0.00/825 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/27.8k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/3.95G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/243 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

In [25]:
def generate_story(name, genre, max_words=1000):
    prompt = f"Write a {genre} story of around {max_words} words with a main character named {name}."
    
    # Tokenize the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # Generate tokens
    output = model.generate(
        **inputs,
        max_new_tokens=1000,  # adjust based on word/token ratio
        do_sample=True,
        temperature=0.8,
        top_p=0.95,
        repetition_penalty=1.1,
        eos_token_id=tokenizer.eos_token_id
    )
    
    # Decode and format output
    story = tokenizer.decode(output[0], skip_special_tokens=True)
    return story

query sent to the LLM

In [26]:
# Example usage
name = selected_users.iloc[0]["given_name"]
genre = selected_users.iloc[0]["genre"]

story = generate_story(name, genre)
print(story)

Write a Romance story of around 1000 words with a main character named Michael. Include one or more supporting characters and set it in a city setting. Ensure there are conflicts within the story and include resolution.

Title: The City of Love

Chapter One - A New Beginning
Michael was the type who always had a plan for everything. He would go to college, get a good job, buy a nice house, and raise a family with his wife. That’s how he saw life – one step at a time. But as soon as he arrived in New York City, all of those plans went out the window.
He had just finished unpacking his last box when the doorbell rang. Michael opened the door to find a young woman standing on his doorstep. She was wearing an oversized sweatshirt that hid most of her figure, but she was unmistakably beautiful. Her long black hair fell freely down her back, and her big brown eyes seemed to sparkle.
"Um... hello," Michael said, feeling slightly flustered by her sudden appearance.
"Hi, sorry to bother you," s

In [22]:
name, genre

('Michael', 'Romance')

<h2>Text-to-Speech</h2>

In [27]:
!pip install TTS

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.7/3.7 MB 60.0 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 101.7 MB/s eta 0:00:0000:010:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 58.4 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.9/10.9 MB 109.4 MB/s eta 0:00:0000:010:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 92.6 MB/s eta 0:00:00:00:0100:01
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of librosa to 

In [29]:
from TTS.api import TTS

# List available models

# Load a lightweight model (e.g., tacotron2 + vocoder)
tts = TTS(model_name="tts_models/en/ljspeech/tacotron2-DDC", progress_bar=False, gpu=False)
tts.tts_to_file(text=story+"....continue your streak to read further.", file_path="output.wav")


 > Downloading model to /root/.local/share/tts/tts_models--en--ljspeech--tacotron2-DDC
 > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Downloading model to /root/.local/share/tts/vocoder_models--en--ljspeech--hifigan_v2
 > Model's license - apache 2.0
 > Check https://choosealicense.com/licenses/apache-2.0/ for more info.
 > Using model: Tacotron2
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:True
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linea

/usr/local/lib/python3.10/dist-packages/TTS/utils/io.py:54: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  return torch.load(f, map_location=map_location, **kwargs)


 > Model's reduction rate `r` is set to: 1
 > Vocoder Model: hifigan
 > Setting up Audio Processor...
 | > sample_rate:22050
 | > resample:False
 | > num_mels:80
 | > log_func:np.log
 | > min_level_db:-100
 | > frame_shift_ms:None
 | > frame_length_ms:None
 | > ref_level_db:20
 | > fft_size:1024
 | > power:1.5
 | > preemphasis:0.0
 | > griffin_lim_iters:60
 | > signal_norm:False
 | > symmetric_norm:True
 | > mel_fmin:0
 | > mel_fmax:8000.0
 | > pitch_fmin:1.0
 | > pitch_fmax:640.0
 | > spec_gain:1.0
 | > stft_pad_mode:reflect
 | > max_norm:4.0
 | > clip_norm:True
 | > do_trim_silence:False
 | > trim_db:60
 | > do_sound_norm:False
 | > do_amp_to_db_linear:True
 | > do_amp_to_db_mel:True
 | > do_rms_norm:False
 | > db_level:None
 | > stats_path:None
 | > base:2.718281828459045
 | > hop_length:256
 | > win_length:1024
 > Generator Model: hifigan_generator
 > Discriminator Model: hifigan_discriminator
Removing weight norm...
 > Text splitted to sentences.
['Write a Romance story of around 

'output.wav'